In [1]:
import ai2thor
import copy
import ai2thor.controller
BASE_DIR = "/Users/jiasenl/Code/ai2thor/unity/builds/thor-local-OSXIntel64.app/Contents/MacOS/AI2-Thor"
# BASE_DIR = '/home/jiasen/Code/ai2thor/unity/builds/thor-my_great_build-Linux64'
controller = ai2thor.controller.Controller(local_executable_path=BASE_DIR, scene='FloorPlan3_physics', gridSize=0.25, agentMode='arm', agentControllerType='mid-level', width=1000, height=1000)


Initialize return: {'cameraNearPlane': 0.1, 'cameraFarPlane': 20.0}


In [2]:
event = controller.step(action='LookDown', degrees=30)

In [3]:
def move_to_somewhere(base_position):
    event = controller.step(action='MoveMidLevelArmHeight', y=base_position['h'], speed = 1.0, returnToStart = False, stopArmMovementOnContact = True)
    print(event.metadata['lastActionSuccess'])
    event = controller.step(action='MoveMidLevelArm', position=dict(x=base_position['x'], y=base_position['y'], z=base_position['z']), speed = 1.0, returnToStart = False, handCameraSpace = False, stopArmMovementOnContact = True)
    print(event.metadata['lastActionSuccess'])
    
# Move the hand to init position
base_position = {'x':0.2, 'y': 0, 'z': 0.2, 'h': 0.6}
move_to_somewhere(base_position)

True
True


In [4]:

def move(command, base_position, rotation, event):
    change_value = 0.05
    change_height = 0.05
    prev_position = copy.copy(base_position)
    if command == 'uz':
        base_position['z'] += change_value
    elif command == 'dz':
        base_position['z'] -= change_value
    elif command == 'ux':
        base_position['x'] += change_value
    elif command == 'dx':
        base_position['x'] -= change_value
    elif command == 'uy':
        base_position['y'] += change_value
    elif command == 'dy':
        base_position['y'] -= change_value
    elif command == 'uh':
        base_position['h'] += change_height
    elif command == 'dh':
        base_position['h'] -= change_height
    elif command == 'rx':
        rotation['x'] += 20
    elif command == 'ry':
        rotation['y'] += 20
    elif command == 'rz':
        rotation['z'] += 20
    elif command == 'cx':
        rotation['x'] -= 20
    elif command == 'cy':
        rotation['y'] -= 20
    elif command == 'cz':
        rotation['z'] -= 20
    elif command == 'd':
        event = controller.step(action='DropMidLevelHand')
        print(event.metadata['lastActionSuccess'])
    elif command == 'm':
        event = controller.step(action='MoveAhead')
        print('move', event.metadata['lastActionSuccess'])
    elif command == 'r':
        event = controller.step(action='RotateRight',degrees=45)
        print('move', event.metadata['lastActionSuccess'])
    elif command == 'l':
        event = controller.step(action='RotateLeft',degrees=45)
        print('move', event.metadata['lastActionSuccess'])
    elif command == 'p':
        event = try_pickup()

    if command in ['uz', 'dz', 'ux', 'dx', 'uy', 'dy']:

        if len(event.metadata['arm']['HeldObjects']) == 0:
            stopArmMovementOnContact = False
        else:
            stopArmMovementOnContact = False
        
        print(stopArmMovementOnContact)
        event = controller.step(action='MoveMidLevelArm', 
                                position=dict(x=base_position['x'], y=base_position['y'], z=base_position['z']), 
                                speed=1.0,
                                returnToStart=False,
                                handCameraSpace=False,
                                stopArmMovementOnContact=stopArmMovementOnContact)
        
        success = event.metadata['lastActionSuccess']
    elif command in ['uh', 'dh']:
        event = controller.step(action='MoveMidLevelArmHeight', y=base_position['h'], speed = 1.0, returnToStart = False, stopArmMovementOnContact = True)
        success = event.metadata['lastActionSuccess']
    elif command in ['rx', 'ry', 'rz', 'cx', 'cy', 'cz']:
        event = controller.step(action='RotateMidLevelHand', rotation=rotation, timeStep = 1.0, returnToStart=False)
        success = event.metadata['lastActionSuccess']
    elif command in ['r', 'm', 'l', 'p']:
        success = event.metadata['lastActionSuccess']

    if not success:
        base_position = prev_position
    
    print('success', success)
    print(base_position)
    event = controller.step(action='WhatObjectsCanHandPickUp')
    print(event.metadata['actionReturn'])
    return event, base_position, rotation

def try_pickup():
    event = controller.step(action='PickUpMidLevelHand')
    print(event.metadata['lastActionSuccess'])

    return event

def move_by_sequence(command_sequence, base_position, rotation, event):
    for command in command_sequence:
        print(command)
        event, base_position, rotation = move(command, base_position, rotation, event)
    return event, base_position, rotation

def move_manually(base_position, rotation, event):
    change_value = 0.05
    change_height = 0.05
    base_position =  {'x':0.2, 'y': 0, 'z': 0.2, 'h': 0.6}
    rotation = {'x':0, 'y': -90, 'z': 0}
    set_of_all_actions = []
    while(True):
        command = input()
        if command == 'q':
            print(set_of_all_actions)
            break
        event, base_position, rotation = move(command, base_position, rotation, event)
        set_of_all_actions.append(command)
    

In [5]:
seq = ['r', 'r', 'r', 'r', 'r', 'r', 'm', 'm', 'm', 'm', 
       'm', 'm', 'm', 'm', 'm', 'm', 'l', 'r', 'r', 'r', 
       'm', 'm', 'dx', 'dx', 'dx', 'dx', 'dx', 'dx', 'dx', 
       'dy', 'dy', 'dy', 'uz', 'uz', 'uz', 'dy', 'uz', 'dy', 
       'uz', 'rx', 'rx', 'ux', 'dy', 'uz', 'uz', 'dx', 'dx', 
       'dy', 'p', 'uy', 'uy', 'uy', 'uy', 'uy', 'uy', 'uy', 
       'uy', 'uy','r', 'r', 'm', 'm', 'm', 'm', 'r', 'r', 'm', 'dy', 'dy']

base_position =  {'x':0.2, 'y': 0, 'z': 0.2, 'h': 0.6}
rotation = {'x':0, 'y': -90, 'z': 0}
event = None
event, base_position, rotation = move_by_sequence(seq, base_position, rotation, event)

r
move True
success True
{'x': 0.2, 'y': 0, 'z': 0.2, 'h': 0.6}
[]
r
move True
success True
{'x': 0.2, 'y': 0, 'z': 0.2, 'h': 0.6}
[]
r
move True
success True
{'x': 0.2, 'y': 0, 'z': 0.2, 'h': 0.6}
[]
r
move True
success True
{'x': 0.2, 'y': 0, 'z': 0.2, 'h': 0.6}
[]
r
move True
success True
{'x': 0.2, 'y': 0, 'z': 0.2, 'h': 0.6}
[]
r
move True
success True
{'x': 0.2, 'y': 0, 'z': 0.2, 'h': 0.6}
[]
m
move True
success True
{'x': 0.2, 'y': 0, 'z': 0.2, 'h': 0.6}
[]
m
move True
success True
{'x': 0.2, 'y': 0, 'z': 0.2, 'h': 0.6}
[]
m
move True
success True
{'x': 0.2, 'y': 0, 'z': 0.2, 'h': 0.6}
[]
m
move True
success True
{'x': 0.2, 'y': 0, 'z': 0.2, 'h': 0.6}
[]
m
move True
success True
{'x': 0.2, 'y': 0, 'z': 0.2, 'h': 0.6}
[]
m
move True
success True
{'x': 0.2, 'y': 0, 'z': 0.2, 'h': 0.6}
[]
m
move True
success True
{'x': 0.2, 'y': 0, 'z': 0.2, 'h': 0.6}
[]
m
move True
success True
{'x': 0.2, 'y': 0, 'z': 0.2, 'h': 0.6}
[]
m
move True
success True
{'x': 0.2, 'y': 0, 'z': 0.2, 'h': 0.6

In [6]:
# show visible object: 
def show_visible_object(event):
    for o in event.metadata['objects']:
        if o['visible']:
            print(o['objectId'])

In [16]:
show_visible_object(event)

CounterTop|-00.08|+01.15|00.00
Floor|+00.00|+00.00|+00.00
Book|+00.15|+01.10|+00.62
Bread|-00.52|+01.17|-00.03
Tomato|-00.39|+01.14|-00.81
Bowl|+00.27|+01.10|-00.75
ButterKnife|-00.41|+01.10|-00.46


In [18]:
event = controller.step('GetSpawnCoordinatesAboveReceptacle', objectId='CounterTop|-00.08|+01.15|00.00', anywhere=True)

In [19]:
event.metadata['actionReturn']

[{'x': 0.493681252, 'y': 1.2046, 'z': 1.0100956},
 {'x': 0.493681252, 'y': 1.2046, 'z': 0.908986032},
 {'x': 0.493681252, 'y': 1.2046, 'z': 0.807876468},
 {'x': 0.493681252, 'y': 1.2046, 'z': 0.7067669},
 {'x': 0.493681252, 'y': 1.2046, 'z': 0.605657339},
 {'x': 0.493681252, 'y': 1.2046, 'z': 0.5045478},
 {'x': 0.493681252, 'y': 1.2046, 'z': 0.4034382},
 {'x': 0.493681252, 'y': 1.2046, 'z': 0.302328646},
 {'x': 0.493681252, 'y': 1.2046, 'z': 0.201219082},
 {'x': 0.493681252, 'y': 1.2046, 'z': 0.100109518},
 {'x': 0.493681252, 'y': 1.2046, 'z': -0.00100004673},
 {'x': 0.493681252, 'y': 1.2046, 'z': -0.102109671},
 {'x': 0.493681252, 'y': 1.2046, 'z': -0.203219175},
 {'x': 0.493681252, 'y': 1.2046, 'z': -0.3043288},
 {'x': 0.493681252, 'y': 1.2046, 'z': -0.4054383},
 {'x': 0.493681252, 'y': 1.2046, 'z': -0.5065479},
 {'x': 0.493681252, 'y': 1.2046, 'z': -0.607657433},
 {'x': 0.493681252, 'y': 1.2046, 'z': -0.708767056},
 {'x': 0.493681252, 'y': 1.2046, 'z': -0.809876561},
 {'x': 0.493681

In [5]:
base_position =  {'x':0.2, 'y': 0, 'z': 0.2, 'h': 0.6}
rotation = {'x':0, 'y': -90, 'z': 0}
event = None
move_manually(base_position, rotation, event)

m
move True
success True
{'x': 0.2, 'y': 0, 'z': 0.2, 'h': 0.6}
[]
m
move True
success True
{'x': 0.2, 'y': 0, 'z': 0.2, 'h': 0.6}
[]
m
move True
success True
{'x': 0.2, 'y': 0, 'z': 0.2, 'h': 0.6}
[]
m
move True
success True
{'x': 0.2, 'y': 0, 'z': 0.2, 'h': 0.6}
[]
r
move True
success True
{'x': 0.2, 'y': 0, 'z': 0.2, 'h': 0.6}
[]
m
move False
success False
{'x': 0.2, 'y': 0, 'z': 0.2, 'h': 0.6}
[]
r
move True
success True
{'x': 0.2, 'y': 0, 'z': 0.2, 'h': 0.6}
[]
m
move True
success True
{'x': 0.2, 'y': 0, 'z': 0.2, 'h': 0.6}
[]
m
move True
success True
{'x': 0.2, 'y': 0, 'z': 0.2, 'h': 0.6}
[]
l
move True
success True
{'x': 0.2, 'y': 0, 'z': 0.2, 'h': 0.6}
[]
l
move True
success True
{'x': 0.2, 'y': 0, 'z': 0.2, 'h': 0.6}
[]
uz
False
success True
{'x': 0.2, 'y': 0, 'z': 0.25, 'h': 0.6}
[]
uz
False
success True
{'x': 0.2, 'y': 0, 'z': 0.3, 'h': 0.6}
[]
uz
False
success True
{'x': 0.2, 'y': 0, 'z': 0.35, 'h': 0.6}
[]
uz
False
success True
{'x': 0.2, 'y': 0, 'z': 0.39999999999999997, 

In [12]:
seq = ['m', 'm', 'm', 'm', 'r', 'm', 'r', 'm', 'm', 'l', 'l']
base_position =  {'x':0.2, 'y': 0, 'z': 0.2, 'h': 0.6}
rotation = {'x':0, 'y': -90, 'z': 0}
event = None
event, base_position, rotation = move_by_sequence(seq, base_position, rotation, event)

m
move True
success True
{'x': 0.2, 'y': 0, 'z': 0.2, 'h': 0.6}
[]
m
move True
success True
{'x': 0.2, 'y': 0, 'z': 0.2, 'h': 0.6}
[]
m
move True
success True
{'x': 0.2, 'y': 0, 'z': 0.2, 'h': 0.6}
[]
m
move True
success True
{'x': 0.2, 'y': 0, 'z': 0.2, 'h': 0.6}
[]
r
move True
success True
{'x': 0.2, 'y': 0, 'z': 0.2, 'h': 0.6}
[]
m
move False
success False
{'x': 0.2, 'y': 0, 'z': 0.2, 'h': 0.6}
[]
r
move True
success True
{'x': 0.2, 'y': 0, 'z': 0.2, 'h': 0.6}
[]
m
move True
success True
{'x': 0.2, 'y': 0, 'z': 0.2, 'h': 0.6}
[]
m
move True
success True
{'x': 0.2, 'y': 0, 'z': 0.2, 'h': 0.6}
[]
l
move True
success True
{'x': 0.2, 'y': 0, 'z': 0.2, 'h': 0.6}
[]
l
move True
success True
{'x': 0.2, 'y': 0, 'z': 0.2, 'h': 0.6}
[]


In [19]:
base_position =  {'x':0.2, 'y': 0.3, 'z': 0.2, 'h': 0.6}
move_to_somewhere(base_position)

True
True


In [13]:
event.metadata['agent']

{'name': 'agent',
 'position': {'x': -1.0, 'y': 1.123206, 'z': -1.25},
 'rotation': {'x': 0.0, 'y': 270.0, 'z': 0.0},
 'cameraHorizon': 30.0000038,
 'isStanding': True,
 'inHighFrictionArea': False}